In [ ]:
import os
import pickle
import sys
import math
sys.path.append("../")
from od.env.config import SUMO_SIM_SECONDS, NET_TIMEOUT_SECONDS, SUMO_TOTAL_SECONDS
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType, BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS, ROOT_DIR
from od.env.station import BS_PRESET
from od.social.manager.types import DynamicSocialGroupBehaviour
import od.misc.logger
import od.misc.statistic as oms
from datetime import datetime
from os.path import exists
DEST_FOLDER =  "../data/POST_REGNA_PERFECT/"



In [ ]:
class FakeBaseStation:
    def __init__(self, name, pos, bs_type, serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial

    def __repr__(self) -> str:
        return self.name

    def __index__(self):
        return self.serial


In [ ]:
def Run(statistic, interest_config,save_path):
    statistic.nft_traffic = {}
    statistic.interest_config = interest_config
    statistic.dirpath = save_path
    BS_CONFIG = {
        name: bs_config
        for name, bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA or
            (
                interest_config.req_rsu and
                bs_config["type"] == BaseStationType.UMI
            )
        )
    }
    time_text = datetime.now().strftime("%Y-%m-%d %H-%M-%S")

    class GV:
        NET_STATION_CONTROLLER = [
            FakeBaseStation(
                name,
                BS_CONFIG[name]["pos"],
                BS_CONFIG[name]["type"],
                i
            ) for i, name in enumerate(BS_CONFIG)
        ]
        STATISTIC = od.misc.logger.Logger(statistic.dirpath, "Statistic ({}).txt".format(time_text))
        RESULT = od.misc.logger.Logger(statistic.dirpath, "Result ({}).txt".format(time_text))
    oms.GV = GV
#############################################################################################################
    # preprocess overtimes
#############################################################################################################
    statistic.Report(True)
    GV.STATISTIC.Encapsulate()
    GV.RESULT.Encapsulate()


In [ ]:
# interest_config = InterestConfig(DynamicSocialGroupBehaviour.MAX_N_MEMBER,20, False, ResourceAllocatorType.NOMA_OPT, True,1.4, 1)
# old_statistic = None
# with open("../data/REGNA_PERFECT/{}/object.pickle".format(interest_config.folder()), "rb") as file:
#     old_statistic = pickle.load(file)
# Run(old_statistic, interest_config)

In [ ]:
for res_alloc_type in [ResourceAllocatorType.NOMA_APR,ResourceAllocatorType.NOMA_OPT]:
    for rsu in [False, True]:
        for qos_re_class in [True, False] if rsu else [False]:
            for traffic_scale in [i / 10 for i in range(10, 15, 1)]:
                for seed in [i + 1 for i in range(20)]:
                    interest_config = InterestConfig(DynamicSocialGroupBehaviour.MAX_N_MEMBER,20, qos_re_class, res_alloc_type, rsu, traffic_scale, seed)
                    old_statistic = None
                    save_path = DEST_FOLDER + interest_config.folder()
                    if(not exists(save_path+"report.pickle")):
                        print(save_path)
                        with open("../data/REGNA_PERFECT/{}/object.pickle".format(interest_config.folder()), "rb") as file:
                            old_statistic = pickle.load(file)
                        Run(old_statistic, interest_config,save_path)
                    else:
                        print("{}, Post Process Exists, Skipping...".format(save_path+"report.pickle"))


In [ ]:
for dyn_sg_behav in DynamicSocialGroupBehaviour:
    for dyn_sg_conf in [(i*2+5) for i in range(5)]:
        for qos_re_class in [True ,False]:
            for seed in [i + 1 for i in range(20)]:        
                interest_config = InterestConfig(dyn_sg_behav,dyn_sg_conf, qos_re_class, ResourceAllocatorType.NOMA_APR, True, 1.4, seed)
                old_statistic = None
                save_path = DEST_FOLDER + interest_config.folder()
                if(not exists(save_path+"report.pickle")):
                    print(save_path)
                    with open("../data/REGNA_PERFECT/{}/object.pickle".format(interest_config.folder()), "rb") as file:
                        old_statistic = pickle.load(file)
                    Run(old_statistic, interest_config,save_path)
                else:
                    print("{}, Post Process Exists, Skipping...".format(save_path+"report.pickle"))